## Notebook Settings



In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run /home/leon/models/lif_cpp/notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Python exe
/home/leon/mambaforge/envs/dual_data/bin/python

## Imports



In [1]:
import sys
sys.path.insert(0, '/home/leon/models/lif_cpp')  
REPO_PATH = "/home/leon/models/lif_cpp"

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import subprocess
import pandas as pd
from time import sleep
from yaml import safe_load
from scipy.stats import circmean
from joblib import Parallel, delayed

from run_model import run_cpp, update_conf, restore_conf
from analysis.decode import decode_bump, circcvl

## Helpers



In [1]:
def get_precision(x):

    cmean =  (x - circmean(x, low=-np.pi, high=np.pi)) % (2* np.pi) - np.pi

    cmean[cmean > np.pi/2] -= np.pi
    cmean[cmean < -np.pi/2] += np.pi

    return cmean

In [1]:
def get_data(session='/'):
  # Open the file in binary mode and read the data
  with open('/home/leon/models/lif_cpp/data/simul/'+ session + '/rates.txt', 'rb') as f:
    data = np.fromfile(f, dtype=np.float32)
  # Now 'data' is a numpy array containing your data
  return data.reshape(-1, 40000).T

In [1]:
def get_rate(name, ini, phi):
     session = "/%s_phi_%d_ini_%d/" % (name, phi, ini)
     rate = get_data(session)
     return rate[:32000]

def get_rates_ini_phi(name, ini_list, phi_list):
     rates = Parallel(n_jobs=-1)(delayed(get_rate)(name, ini, phi) for ini in ini_list for phi in phi_list)
     rates = np.array(rates)
     rates = rates.reshape(len(ini_list), len(phi_list), 32000, -1)
     return rates

In [1]:
def get_df_ini_phi(rates):
    n_trials, n_phi, n_neurons, n_times = rates.shape

    # Create indices
    trials_ind, phi_ind, neurons_ind, times_ind = np.indices((n_trials, n_phi, n_neurons, n_times))

    # Construct DataFrame
    df = pd.DataFrame({
        'trial': trials_ind.flatten(),
        'phi': phi_ind.flatten(),
        'neuron': neurons_ind.flatten(),
        'time': times_ind.flatten(),
        'rates': rates.flatten()
    })

    return df

In [1]:
def load_data_ini_phi(name, ini_list, phi_list):
    rates = get_rates_ini_phi(name, ini_list, phi_list)
    df = get_df_ini_phi(rates)
    return df

In [1]:
def get_code(df):
    df_code = df.groupby(['time', 'trial', 'phi'])['rates'].apply(decode_bump).reset_index()
    df_code[['m0', 'm1', 'phase']] = pd.DataFrame(df_code['rates'].tolist(), index=df_code.index)
    df_code = df_code.drop(columns=['rates'])

    end_point = df_code[df_code.time==df_code.time.iloc[-1]]
    end_point = end_point.drop(columns=['time'])
    print(end_point.head())  
    return df_code, end_point

In [1]:
def run_ini_phi(conf_name, name, ini_list, phi_list, Ie=.004, Jab=0, axis=0):
    restore_conf(REPO_PATH + '/conf/'+ conf_name +'.yml',
                 REPO_PATH + '/conf/'+ conf_name + '.yml.bak')

    conf_path = REPO_PATH + '/conf/'+ conf_name +'.yml'

    for ini in ini_list:
        for phi in phi_list:
            session = "%s_phi_%d_ini_%d" % (name, phi, ini)
            data_path = REPO_PATH + '/data/simul/' + session
            update_conf(REPO_PATH + '/conf/' + conf_name, 'DATA_PATH', data_path)
            update_conf(REPO_PATH + '/conf/' + conf_name, 'PHI_STIM', float(phi), axis=0)
            update_conf(REPO_PATH + '/conf/' + conf_name, 'Iext', float(Ie), axis=0)
            if Jab>0:
                update_conf(REPO_PATH + '/conf/' + conf_name, 'Jab', float(Jab), axis=axis)

            sleep(.2)
            run_cpp(session, conf_path=conf_path)
            sleep(.2)

            subprocess.run([REPO_PATH + '/src/mem_usage.sh'])
            subprocess.run([REPO_PATH + '/src/cpu_usage.sh'])

    restore_conf(REPO_PATH + '/conf/' + conf_name + '.yml.bak',
                 REPO_PATH + '/conf/' + conf_name + '.yml')

## Multiple Trials



#### Simulation



##### Parameters



In [1]:
config = safe_load(open("/home/leon/models/lif_cpp/conf/config_ODR.yml", "r"))
conf_name = 'config_ODR_multi'
ini_list = np.arange(0, 10)
phi_list = np.linspace(0, 315, 8)
# phi_list = [180]

##### Control



In [1]:
run_ini_phi(conf_name, 'odr_off', ini_list, phi_list, Ie=config['Iext'][0])

File moved successfully!
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
File moved successfully!

##### NB ON



In [1]:
run_ini_phi(conf_name, 'odr_on', ini_list, phi_list, Ie=config['Iext'][0]*1.2)

File moved successfully!
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
File moved successfully!

##### Jee



In [1]:
run_ini_phi(conf_name, 'odr_Jee', ini_list, phi_list, Ie=config['Iext'][0] * 1.2, Jab=config['Jab'][0] * 1.2)

File moved successfully!
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
File moved successfully!

##### Jei



In [1]:
run_ini_phi(conf_name, 'odr_Jei', ini_list, phi_list, Ie=config['Iext'][0] * 1.2, Jab=config['Jab'][1] * 0.9, axis=1)

File moved successfully!
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
 MEM_USAGE > 85.0%, sleeping for a while ...
File moved successfully!

#### Analysis



In [1]:
df = load_data_ini_phi('odr_off', ini_list, phi_list)
df_code, end_point = get_code(df)

trial  phi        m0        m1     phase
3040      0    0  0.500375  0.074867  6.225282
3041      0    1  0.501125  0.036558  6.209045
3042      0    2  0.497500  0.032575  5.093944
3043      0    3  0.488125  0.046029  4.430990
3044      0    4  0.495375  0.021653  3.442677

In [1]:
df_on = load_data_ini_phi('odr_Jee', ini_list, phi_list)
df_code_on, end_point_on = get_code(df_on)

trial  phi         m0        m1     phase
3040      0    0   9.820250  0.425701  1.238873
3041      0    1  10.042500  0.517968  1.011137
3042      0    2  10.059875  0.683268  0.578462
3043      0    3  10.146750  0.306271  0.414573
3044      0    4   9.943875  0.397438  6.090778

#### Phases



In [1]:
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

sns.lineplot(data=df_code, x='time', y=df_code['phase']*180/np.pi, legend=False, lw=2, ax=ax[0], hue='phi', alpha=0.25)

sns.lineplot(data=df_code_on, x='time', y=df_code_on['phase']*180/np.pi, legend=False, lw=2, ax=ax[0], hue='phi', alpha=0.25)

ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('$\phi$ (°)')
# ax[0].set_xticks([0, 1, 2, 3, 4, 5])
ax[0].set_yticks([0, 90, 180, 270, 360])

sns.histplot(data=end_point, x=end_point['phase']*180/np.pi, legend=False, ax=ax[1], bins=200, kde=False, stat='density')

sns.histplot(data=end_point_on, x=end_point_on['phase']*180/np.pi, legend=False, ax=ax[1], bins=200, kde=False, stat='density')

ax[1].set_xlabel('$\phi$ (°)')
ax[1].set_ylabel('$Count$')
ax[1].set_xticks([0, 90, 180, 270, 360])
plt.show()

#### Endpoint Errors



In [1]:
end_point['accuracy'] = (end_point.phase - end_point['phi'] / 180 * np.pi) % (2 * np.pi)
end_point['precision'] = end_point.groupby('phi')['phase'].transform(get_precision)

end_point_on['accuracy'] = (end_point_on.phase - end_point_on['phi'] / 180 * np.pi) % (2 * np.pi)
end_point_on['precision'] = end_point_on.groupby('phi')['phase'].transform(get_precision)
print(end_point.head())

trial  phi        m0        m1     phase  accuracy  precision
3040      0    0  0.500375  0.074867  6.225282  6.225282  -0.099011
3041      0    1  0.501125  0.036558  6.209045  6.191592   0.330141
3042      0    2  0.497500  0.032575  5.093944  5.059038   0.008580
3043      0    3  0.488125  0.046029  4.430990  4.378630   0.082864
3044      0    4  0.495375  0.021653  3.442677  3.372864   0.433607

In [1]:
fig, ax = plt.subplots(1, 3, figsize=[2*width, height])

sns.histplot(data=end_point, x=end_point['phase']*180/np.pi, legend=False, lw=2, ax=ax[0], kde=False, bins=200, stat='density')
sns.histplot(data=end_point_on, x=end_point_on['phase']*180/np.pi, legend=False, lw=2, ax=ax[0], kde=False, bins=200, stat='density')
ax[0].set_xlabel('$\phi$(°)')
ax[0].set_ylabel('Density')
ax[0].set_xticks([0, 90, 180, 270, 360])

sns.histplot(data=end_point, x=end_point['accuracy']*180/np.pi, legend=False, lw=2, ax=ax[1], kde=False, bins=200, stat='density')
sns.histplot(data=end_point_on, x=end_point_on['accuracy']*180/np.pi, legend=False, lw=2, ax=ax[1], kde=False, bins=200, stat='density')
ax[1].set_xlabel('$\phi - \phi_{stim}$ (°)')
ax[1].set_ylabel('Density')
ax[1].set_xticks([0, 90, 180, 270, 360])

sns.histplot(data=end_point, x=end_point['precision']*180/np.pi, legend=False, ax=ax[2], bins='auto', kde=False, stat='density', element='step', alpha=0)
sns.histplot(data=end_point_on, x=end_point_on['precision']*180/np.pi, legend=False, ax=ax[2], bins='auto', kde=False, stat='density', element='step', alpha=0.)
ax[2].set_xlabel('$\phi - <\phi>_{trials}$ (°)')
ax[2].set_ylabel('Density')
ax[2].set_xlim([-10, 10])

plt.show()